In [1]:
import joblib
from tensorflow.keras.models import load_model

In [3]:
# Load models
arima_model = joblib.load("arima_model.pkl")
sarima_model = joblib.load("sarima_model.pkl")
lstm_model = load_model("lstm_model.keras")

c:\Users\Firew Ayele\Desktop\kifiya\GMF_Investments\.venv_gmf\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
#  use for forecasting ARIMA, SARIMA,
model = arima_model  

# Forecast parameters
forecast_steps = 180  # For a 6-month forecast (assuming 30 trading days per month)


In [5]:
# Forecast with ARIMA 
def forecast_arima_sarima(model, steps):
    forecast = model.get_forecast(steps=steps)
    forecast_mean = forecast.predicted_mean
    confidence_intervals = forecast.conf_int()
    return forecast_mean, confidence_intervals